In [24]:
from arcgis.gis import GIS
#from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayerCollection
import pandas as pd
import arcpy, os, time, uuid
from zipfile import ZipFile
import arcgis.features
import sys, os, shutil

# Start Timer
startTime = time.time()

gis =GIS('PRO')

existe=True
lst=pd.read_html('https://www.estar.cl/fire/planilla.php')
df=pd.DataFrame(lst[0],columns=['TEMPORADA',	'INICIO',	'SEMANA INICIO',	'SEMANA KPI',	'ANO INICIO',	'MES INICIO',	'DIA INICIO',	'HORA INICIO',	'COD_SEMANA',	'DIA_SEMANA',	'RANGO_PREV',	'REGION',	'PROVINCIA',	'COMUNA',	'EMPRESA',	'PROPIETARIO',	'TIPO',	'AREA',	'FICHA',	'COD_PREDIO',	'PREDIO',	'FOCOS',	'MULTIFOCO',	'DANO PLANTAC',	'CAUSA',	'CODIGO_CAUSA',	'CONFLICTO',	'GRUPO CAUSAL',	'ZONA META',	'COD META',	'COMBUSTIBLE INICIAL',	'COMBUSTIBLE FINAL',	'DESPACHADOR',	'DETECTOR',	'AVISO',	'COMANDANTE',	'ANCHO LINEA',	'LARGO LINEA',	'CLASE',	'GASTO COMBATE UF',	'FECHA CONTROL',	'HORA CONTROL',	'LONGITUD',	'LATITUD',	'ESTADO',	'SUP INICIAL',	'PIRA',	'EUCA',	'OTRAS',	'TOTAL',	'SUM DANO',	'USD DANO',	'ESTADO DANO',	'DETECCION',	'DESPACHO',	'ARRIBO',	'CONTENCION',	'CONTROL',	'EXTINCION',	'MIN DESPACHO',	'PROM SALIDA',	'MIN ARRIBO',	'PROM INICIO COMBATE',	'MIN CONTENCION',	'MIN CONTROL',	'MIN EXTINCION',	'MIN HASTA CONTENCION',	'MIN TOTAL',	'RECURSOS DESPACHO',	'RECURSOS COMBATE',	'RECURSOS CONTROL',	'ESTIMADO CONTROL',	'AREA CONTROL'])
# Convert the values to real coordinate values
df['LONGITUD'] = df['LONGITUD'].astype(str).str[:3] + '.' + df['LONGITUD'].astype(str).str[3:]
df['LATITUD'] = df['LATITUD'].astype(str).str[:3] + '.' + df['LATITUD'].astype(str).str[3:]


# Define a lambda function to apply the logic
def get_part_of_day(hour):
    if hour < 7:
        return "Noche"
    elif hour < 12:
        return "Mañana"
    elif hour < 21:
        return "Tarde"
    else:
        return "Noche"

# Apply the lambda function to create the new column "Parte del día"
df["Partedia"] = df["HORA INICIO"].apply(lambda x: get_part_of_day(x))

import pandas as pd
from datetime import datetime

df['DateTime'] = pd.to_datetime(df['INICIO'])

# Extract specific components
df['Date'] = df['DateTime'].dt.date


# Add a new column 'DayOfWeek' with the day of the week (Monday = 1 to Sunday = 7)
df['DayOfWeek'] = df['Date'].apply(lambda x: x.weekday() + 1)
df['ID'] = df.reset_index().index

df_columns=['ID','TEMPORADA', 'INICIO', 'SEMANA KPI'
            , 'EMPRESA', 'AREA', 'FICHA'
            , 'COD_PREDIO', 'PREDIO', 'FOCOS', 'MULTIFOCO'
            , 'CAUSA', 'ESTADO', 'PIRA'
            , 'EUCA', 'OTRAS',"HORA INICIO", 'Partedia', 'DayOfWeek'
            , 'LONGITUD', 'LATITUD']
df_selected=df.loc[:,df_columns]
df_rename={'TEMPORADA': 'Temporada'
           , 'INICIO':'Fecha'
           , 'SEMANA KPI':'Semana'
           , 'EMPRESA':'Zona'
           , 'AREA':'Paisaje'
           , 'FICHA':'Ficha'
           , 'COD_PREDIO':'idpredio'
           , 'PREDIO':'Predio'
           , 'FOCOS':'Focos'
           , 'MULTIFOCO':'Multifocos'
           , 'CAUSA':'Causa'
           , 'ESTADO':'Estado'
           , 'PIRA':'Pira ha'
           , 'EUCA':'Euca ha'
           , 'OTRAS':'Otras ha'
           ,"HORA INICIO":'Hora inicio'
           , 'Partedia':'Parde_del_dia'
            , 'DayOfWeek':'Dia'
            ,'LONGITUD':'longitude'
            ,'LATITUD':'latitude'}

df_selected=df_selected.rename(columns=df_rename)

df3=pd.read_csv('data/lastupdate.csv')
new_records = df_selected[~df_selected['Ficha'].isin(df3['Ficha'])]


In [25]:
sdf=pd.DataFrame.spatial.from_xy(df_selected, x_column='longitude', y_column='latitude')
temp_gdb = 'C:/temp/temp_wildfire.gdb'  # Change this path to your desired temporary location

In [26]:
if os.path.exists(temp_gdb):
    shutil.rmtree(temp_gdb)
else:
    pass

In [22]:
if len(new_records)>0:
    arcpy.management.CreateFileGDB(os.path.dirname(temp_gdb), os.path.basename(temp_gdb))
    sdf.spatial.to_featureclass(location=temp_gdb + '//temp_file', overwrite=True)
else:
    sys.exit()

In [27]:
len(new_records)

33

In [23]:


# # Overwrite Output

# Overwrite Output
arcpy.env.overwriteOutput = True

fc = temp_gdb + '//temp_file'                                   # Path to Feature Class/Table
fsItemId = "d5b63197aea34575970a6b2b05cf1ab6"                   # Feature Service Item ID to update
featureService = True                                           # True if updating a Feature Service, False if updating a Hosted Table
hostedTable = False                                             # True is updating a Hosted Table, False if updating a Feature Service
layerIndex = 0                                                  # Layer Index
disableSync = True                                              # True to disable sync, and then re-enable sync after append, False to not disable sync.  Set to True if sync is not enabled
updateSchema = True                                             # True will remove/add fields from feature service keeping schema in-sync, False will not remove/add fields

# Start Timer
startTime = time.time()

# Create GIS object
print("Connecting to AGOL")
gis = GIS("PRO")

# Create UUID variable for GDB
gdbId = str(uuid.uuid1())

# Function to Zip FGD
def zipDir(dirPath, zipPath):
    '''Zip File Geodatabase'''
    zipf = ZipFile(zipPath , mode='w')
    gdb = os.path.basename(dirPath)
    for root, _ , files in os.walk(dirPath):
        for file in files:
            if 'lock' not in file:
               filePath = os.path.join(root, file)
               zipf.write(filePath , os.path.join(gdb, file))
    zipf.close()

print("Creating temporary File Geodatabase")
gdb = arcpy.CreateFileGDB_management(arcpy.env.scratchFolder, gdbId)[0]

# Export featureService classes to temporary File Geodatabase
fcName = os.path.basename(fc)
fcName = fcName.split('.')[-1]
print(f"Exporting {fcName} to temp FGD")
if featureService == True:
    arcpy.conversion.FeatureClassToFeatureClass(fc, gdb, fcName)
elif hostedTable == True:
    arcpy.conversion.TableToTable(fc, gdb, fcName)

# Zip temp FGD
print("Zipping temp FGD")
zipDir(gdb, gdb + ".zip")

# Upload zipped File Geodatabase
print("Uploading File Geodatabase")
fgd_properties={'title':gdbId, 'tags':'temp file geodatabase', 'type':'File Geodatabase'}
fgd_item = gis.content.add(item_properties=fgd_properties, data=gdb + ".zip")

# Get featureService/hostedTable layer
serviceLayer = gis.content.get(fsItemId)
if featureService == True:
    fLyr = serviceLayer.layers[layerIndex]
elif hostedTable == True:
    fLyr = serviceLayer.tables[layerIndex]

# Truncate Feature Service
# If views exist, or disableSync = False use delete_features.  OBJECTIDs will not reset
flc = arcgis.features.FeatureLayerCollection(serviceLayer.url, gis)
hasViews = False
try:
    if flc.properties.hasViews == True:
        print("Feature Service has view(s)")
        hasViews = True
except:
    hasViews = False

if hasViews == True or disableSync == False:
    objectIdField = fLyr.manager.properties.objectIdField
    # Get Min OBJECTID
    minOID = fLyr.query(out_statistics=[{"statisticType": "MIN", "onStatisticField": objectIdField, "outStatisticFieldName": "MINOID"}])
    minOBJECTID = minOID.features[0].attributes['MINOID']

    # Get Max OBJECTID
    maxOID = fLyr.query(out_statistics=[{"statisticType": "MAX", "onStatisticField": objectIdField, "outStatisticFieldName": "MAXOID"}])
    maxOBJECTID = maxOID.features[0].attributes['MAXOID']

    # If more than 2,000 features, delete in 2000 increments
    if (maxOBJECTID - minOBJECTID) > 2000:
        print("Deleting features")
        x = minOBJECTID
        y = x + 1999
        while x < maxOBJECTID:
            query = f"{objectIdField} >= {x} AND {objectIdField} <= {y}"
            fLyr.delete_features(where=query)
            x += 2000
            y += 2000
    # Else if less than 2,000 features, delete all
    else:
        print("Deleting features")
        fLyr.delete_features(where="1=1")

# If no views and disableSync is True: disable Sync, truncate, and then re-enable Sync.  OBJECTIDs will reset
elif hasViews == False and disableSync == True:
    if flc.properties.syncEnabled == True:
        print("Disabling Sync")
        properties = flc.properties.capabilities
        updateDict = {"capabilities": "Query", "syncEnabled": False}
        flc.manager.update_definition(updateDict)
        print("Truncating Feature Service")
        fLyr.manager.truncate()
        print("Enabling Sync")
        updateDict = {"capabilities": properties, "syncEnabled": True}
        flc.manager.update_definition(updateDict)
    else:
        print("Truncating Feature Service")
        fLyr.manager.truncate()

# Schema Sync
if updateSchema == True:
    # Get feature service fields
    print("Get feature service fields")
    featureServiceFields = {}
    for field in fLyr.manager.properties.fields:
        if field.type != 'esriFieldTypeOID' and 'Shape_' not in field.name and 'GlobalID' not in field.name:
            featureServiceFields[field.name] = field.type

    # Get feature class/table fields
    print("Get feature class/table fields")
    featureClassFields = {}
    arcpy.env.workspace = gdb
    if hostedTable == True:
        for field in arcpy.ListFields(fc):
            if field.type != 'OID' and field.type != 'Geometry' and 'GlobalID' not in field.name:
                featureClassFields[field.name] = field.type
    else:
        for field in arcpy.ListFields(fc):
            if field.type != 'OID' and field.type != 'Geometry' and 'Shape_' not in field.name and 'GlobalID' not in field.name:
                featureClassFields[field.name] = field.type

    minusSchemaDiff = set(featureServiceFields) - set(featureClassFields)
    addSchemaDiff = set(featureClassFields) - set(featureServiceFields)

    # Delete removed fields
    if len(minusSchemaDiff) > 0:
        print("Deleting removed fields")
        for key in minusSchemaDiff:
            print(f"\tDeleting field {key}")
            remove_field = {
                "name": key,
                "type": featureServiceFields[key]
            }
            update_dict = {"fields": [remove_field]}
            fLyr.manager.delete_from_definition(update_dict)

    # Create additional fields
    fieldTypeDict = {}
    fieldTypeDict['Date'] = 'esriFieldTypeDate'
    fieldTypeDict['Double'] = 'esriFieldTypeDouble'
    fieldTypeDict['Integer'] = 'esriFieldTypeInteger'
    fieldTypeDict['String'] = 'esriFieldTypeString'
    if len(addSchemaDiff) > 0:
        print("Adding additional fields")
        for key in addSchemaDiff:
            print(f"\tAdding field {key}")
            if fieldTypeDict[featureClassFields[key]] == 'esriFieldTypeString':
                new_field = {
                    "name": key,
                    "type": fieldTypeDict[featureClassFields[key]],
                    "length": [field.length for field in arcpy.ListFields(fc, key)][0]
                }
            else:
                new_field = {
                    "name": key,
                    "type": fieldTypeDict[featureClassFields[key]]
                }

            update_dict = {"fields": [new_field]}
            fLyr.manager.add_to_definition(update_dict)

# Append features from featureService class/hostedTable
print("Appending features")
#fLyr.append(item_id=fgd_item.id, upload_format="filegdb", upsert=False, field_mappings=[])
arcpy.management.Append(temp_gdb + '//temp_file', "https://araucaria.arauco.com/server/rest/services/Hosted/OcurrenciaIncendiosTActual/FeatureServer/0", "NO_TEST")
# Delete Uploaded File Geodatabase
print("Deleting uploaded File Geodatabase")
fgd_item.delete()

# Delete temporary File Geodatabase and zip file
print("Deleting temporary FGD and zip file")
arcpy.Delete_management(gdb)
arcpy.management.Delete(temp_gdb)
os.remove(gdb + ".zip")

endTime = time.time()
elapsedTime = round((endTime - startTime) / 60, 2)
print("Script finished in {0} minutes".format(elapsedTime))

Connecting to AGOL
Creating temporary File Geodatabase
Exporting temp_file to temp FGD
Zipping temp FGD
Uploading File Geodatabase
Truncating Feature Service
Get feature service fields
Get feature class/table fields
Adding additional fields
	Adding field pira_ha


Exception: ERROR: Field name collision found: pira_ha.
(Error Code: 500)

In [14]:
import ee
import geemap
ee.Authenticate()
#ee.Initialize()

#Map = geemap.Map()


ModuleNotFoundError: No module named 'ee'

In [1]:
%pip install geemap

^C
Note: you may need to restart the kernel to use updated packages.


  Using cached geemap-0.20.4-py2.py3-none-any.whl (2.1 MB)
  Using cached colour-0.1.5-py2.py3-none-any.whl (23 kB)
  Using cached bqplot-0.12.42-py2.py3-none-any.whl (1.2 MB)
  Using cached ipytree-0.2.2-py2.py3-none-any.whl (1.3 MB)
  Using cached xyzservices-2023.2.0-py3-none-any.whl (55 kB)
  Using cached whiteboxgui-2.3.0-py2.py3-none-any.whl (108 kB)
  Using cached folium-0.15.1-py2.py3-none-any.whl (97 kB)
  Using cached sankee-0.2.4-py3-none-any.whl
  Using cached PyCRS-1.0.2-py3-none-any.whl
  Using cached ipyevents-2.0.2-py3-none-any.whl (101 kB)
  Using cached ipyleaflet-0.18.1-py3-none-any.whl (3.7 MB)
  Using cached earthengine-api-0.1.384.tar.gz (261 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cac

In [ ]:
t1 = '2023-12-16'
t2 = '2024-02-24'
roi = ee.FeatureCollection('users/raulperezastorga/Limites_SPPP')

def addNBR_s2h(image):
    nbr = image.normalizedDifference(['B8', 'B12']).rename("nbr")
    image = image.addBands(nbr)
    return image

def mask_fire_firms(image):
    firms = ee.ImageCollection('FIRMS')
    fires_firms = ee.ImageCollection(firms\
            .select('T21')\
            .filterDate(t1,t2)\
            .filterBounds(roi)) 
    fires_firmsmx = fires_firms.max().clip(roi).subtract(273.15)
    fires_firmsrc = fires_firmsmx.gte(100)
    return image.updateMask(fires_firmsrc)

# Define a function to apply the vegetation mask to an input image
def maskVegetation(image):
    mcd12q1 = ee.ImageCollection('MODIS/006/MCD12Q1') \
        .filter(ee.Filter.date(t1, t2)) \
        .filterBounds(roi)\
        .first()
    vg = mcd12q1.select('LC_Type1')
    # Get the pixels that represent vegetation
    mask = vg.eq(1).Or(vg.eq(2)).Or(vg.eq(3)).Or(vg.eq(4)).Or(vg.eq(5)).Or(vg.eq(6)).Or(vg.eq(7))
    
    # Return the masked image with updated metadata
    return image.updateMask(mask)

# mask elevation
def mask_elevation(image):
    elevation_data = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
    elevation_mask = elevation_data.lt(3500)
    return image.updateMask(elevation_mask)


firms = ee.ImageCollection('FIRMS')
fires_firms = ee.ImageCollection(firms\
        .select('T21')\
        .filterDate(t1,t2)\
        .filterBounds(roi)) 
fires_firmsmx = fires_firms.max().clip(roi).subtract(273.15)
fires_firmsrc = fires_firmsmx.gte(100)

In [ ]:

s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterDate(t1, t2) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40)) \
    .filterBounds(roi) \
    .sort('system:time_start', True) \
    .map(lambda image: image.addBands(image.expression('(RED * 1.53)', { 'RED': image.select('B4') }).rename('B4B')))

vis = {
    'bands': ['B8', 'B4B', 'B3'],
    'min': 0,
    'max': 3800,
    'gamma': [1, 1, 1]
}

img = s2.mosaic().clip(roi)
# Use geemap to display the image


In [ ]:
Map.addLayer(img, vis, 'falso')
Map.layers